In [28]:
import pandas as pd
import numpy as np
import lasio

In [59]:
df = pd.read_excel("log.xlsx")
df.columns = df.columns.str.strip().str.upper()
col_units = df.iloc[0].to_dict()
df.drop(0, axis=0, inplace=True)
df = df.apply(pd.to_numeric, errors='coerce')
df

,DEPTH,GR,K,U,TH
1,3837.0,3047,1.697,17.854,62.429
2,3837.1,3343,1.462,17.727,69.207
3,3837.2,3174,1.618,19.406,61.814
4,3837.3,3127,1.305,16.692,61.517
5,3837.4,3024,1.393,18.371,55.971
...,...,...,...,...,...
1767,4016.3,3051,1.697,19.535,58.739
1768,4016.4,3283,1.491,21.088,57.816
1769,4016.5,3107,1.442,19.018,67.965
1770,4016.6,2945,1.530,21.735,64.274


In [67]:
output_file = 'log.las'
depth_name = 'DEPTH'
las = lasio.LASFile()
las.well['WELL'] = 'Volve 15_9-19A'
las.well['NULL'] = -9999.25

for column in df.columns:
    unit = col_units.get(column, 'Unit')
    mnemonic = column.replace('.', '')
    unidad_limpia = unit.replace('.', '')
    descripcion = column
    curve = lasio.CurveItem(mnemonic=mnemonic, unit=unidad_limpia, value=np.array(df[column].values), descr=descripcion)
    las.curves.append(curve)

las.data = df.values.tolist()

if depth_name in df.columns:
    las.well['STRT'] = df[depth_name].min()
    las.well['STOP'] = df[depth_name].max()
    if len(df) > 1:
        las.well['STEP'] = df[depth_name].iloc[1] - df[depth_name].iloc[0]

data_array = df.values.tolist()
las.data = data_array
las.write(output_file, version=2.0)
print(f"Archivo LAS guardado como: {output_file}")

Archivo LAS guardado como: log.las


In [68]:
try:
    f = lasio.read('log.las')
    print("Archivo LAS leído correctamente por lasio:")
    print(f.df())
except Exception as e:
    print(f"Error al leer el archivo LAS con lasio: {e}")

Archivo LAS leído correctamente por lasio:
                                    GR   .C/MIN           [3047 3343 3174 ..  \
DEPTH.M   [3837.  3837.1 3837.2 ..                                             
3837.0                                                                3047.0   
3837.1                                                                3343.0   
3837.2                                                                3174.0   
3837.3                                                                3127.0   
3837.4                                                                3024.0   
...                                                                      ...   
4016.3                                                                3051.0   
4016.4                                                                3283.0   
4016.5                                                                3107.0   
4016.6                                                                2945.0 